# Statistics


**IDEAS**
- SMETANA
    - Find likelyhood that a community member from family X is dependent on a community member in family Y.
    - Find likelyhood that a community member from family X is dependent on a compound A.

- SteadierCOM
    - No statistics?

In [286]:
import math

import pandas as pd
from molmass import Formula
import reframed

In [287]:
import sys
sys.path.append("../functions/")
import general_functions as general_func

import colors_MAGs
import scipy.stats as stats

In [288]:
import numpy as np

### Load universal model

In [289]:
model_uni = reframed.load_cbmodel("/Users/idunmariaburgos/universal_model_extension/output/universe_bacteria.xml")

### Load data 

In [290]:
all_mags_paper = general_func.read_allmags_data()

In [291]:
SC1_C = pd.read_csv("../output/steadiercom_sample_0.1.3/results/results_99_SC1_C.tsv",sep="\t")
SC2_C = pd.read_csv("../output/steadiercom_sample_0.1.3/results/results_99_SC2_C.tsv",sep="\t")
SC1_X = pd.read_csv("../output/steadiercom_sample_0.1.3/results/results_99_SC1_X.tsv",sep="\t")

steadier_sample = pd.concat([SC1_C,SC2_C,SC1_X])

# (steadier_sample.frequency>0.1) & 
steadier_sample = steadier_sample[(steadier_sample.frequency>0.1) & (steadier_sample.rate>1e-6)]

In [292]:
steadier_sample_cross = steadier_sample[(steadier_sample.donor!="environment") & (steadier_sample.receiver!="environment") ].copy()

### Prepare to process data

In [293]:
chebi_lut, chebi_interesting, chebi_colors_ser = colors_MAGs.chebi_rxn_color_func(rxn_based=False)

**Change names of family for readability and better grouping**

In [294]:
all_mags_paper_reduced = all_mags_paper.copy()

all_mags_paper_reduced[all_mags_paper_reduced["new_coverage"]>10][["Source","Substrate","Family","new_coverage"]].sort_values(["Source","Substrate"])

all_mags_paper_reduced["Family"] = all_mags_paper.apply(lambda row: "f_"+row.Family,axis=1)

total_members_family = all_mags_paper_reduced.groupby("Family").count()["Source"].to_dict()
all_mags_paper_reduced["Family"] = all_mags_paper_reduced.apply(lambda row: row.Family if total_members_family[row.Family]>1 else "Other",axis=1)


In [295]:
def mag2family(all_mags_paper):  
    
    family_groups = all_mags_paper.groupby("Family").groups
    mag2family_dict = {mag:family for family,mags in family_groups.items() for mag in mags}
    
    return family_groups,mag2family_dict

**Create dictionaries to translate components into larger groups - MAG-> family, compound-> super_class**

In [296]:
MAGs_steady_com = set(list(steadier_sample_cross.donor.values)+list(steadier_sample_cross.receiver.values))

MAG2sour_sub_id = pd.read_csv("../output/MAG2community_id.tsv",sep="\t",header=None)
MAG2sour_sub_id.columns=["MAG","community_id"]

met2superclass_dict = pd.read_csv("../output/met_chebi_class.tsv",sep="\t",index_col=0)["self defined super class"].to_dict()

family_groups,mag2family_dict = mag2family(all_mags_paper_reduced)

In [297]:
def family_donor(row):
    if row.donor=="environment":
        return "environment"
    else:
        return mag2family_dict[row.donor]


def family_receiver(row):
    if row.receiver=="environment":
        return "environment"
    else:
        return mag2family_dict[row.receiver]


In [298]:
def met2metname(met):
    met_name = model_uni.metabolites[met].name
    return met_name

### Process data

**All compounds**

In [299]:
steadier_sample.loc[:,"family_donor"] = steadier_sample.apply(family_donor,axis=1).copy()
steadier_sample.loc[:,"family_receiver"] = steadier_sample.apply(family_receiver,axis=1).copy()
steadier_sample = steadier_sample[steadier_sample.compound.isin(met2superclass_dict.keys())].copy()
steadier_sample.loc[:,"super_class"] = steadier_sample.apply(lambda x: met2superclass_dict[x.compound],axis=1)

compounds = steadier_sample["compound"].map(met2metname)
steadier_sample.drop("compound",axis=1,inplace=True)
steadier_sample.loc[:,"compound"] = compounds


**Compounds cross-fed**

In [300]:
steadier_sample_cross.loc[:,"family_donor"] = steadier_sample_cross.apply(family_donor,axis=1).copy()
steadier_sample_cross.loc[:,"family_receiver"] = steadier_sample_cross.apply(family_receiver,axis=1).copy()
steadier_sample_cross = steadier_sample_cross[steadier_sample_cross.compound.isin(met2superclass_dict.keys())].copy()
steadier_sample_cross.loc[:,"super_class"] = steadier_sample_cross.apply(lambda x: met2superclass_dict[x.compound],axis=1)

compounds = steadier_sample_cross["compound"].map(met2metname)
steadier_sample_cross.drop("compound",axis=1,inplace=True)
steadier_sample_cross.loc[:,"compound"] = compounds

# FILTERS OUT UNINTERESTING COMPOUNDS
#steadier_sample_cross = steadier_sample_cross[steadier_sample_cross.super_class.isin(chebi_lut.keys())].copy()

steadier_sample_cross["mass_rate*frequency"]=steadier_sample_cross["mass_rate"]*steadier_sample_cross["frequency"]

In [301]:
steadier_sample_cross

,donor,receiver,mass_rate,rate,frequency,community,medium,family_donor,family_receiver,super_class,compound,mass_rate*frequency
13,CH15-bin.0,CH15-bin.7,4.921516e-03,0.027318,0.27,CD_P,SC1_C,f_Treponemataceae,f_Sphaerochaetaceae,simple sugars,D-Glucose,1.328809e-03
33,CH13-bin.11,CH13-bin.4,2.822338e-03,0.008245,0.11,CD_A,SC1_C,f_Desulfovibrionaceae,f_Bacteroidaceae,oligosaccharides,Cellobiose,3.104572e-04
36,CH13-bin.14,CH13-bin.25,2.705991e-03,0.015020,0.20,CD_A,SC1_C,f_Lachnospiraceae,f_Sphaerochaetaceae,simple sugars,D-Glucose,5.411983e-04
39,CH15-bin.7,CH15-bin.8,2.435377e-03,0.013518,0.12,CD_P,SC1_C,f_Sphaerochaetaceae,f_Desulfovibrionaceae,simple sugars,D-Glucose,2.922452e-04
41,CH15-bin.23,CH15-bin.8,2.385105e-03,0.013239,0.17,CD_P,SC1_C,Other,f_Desulfovibrionaceae,simple sugars,D-Glucose,4.054678e-04
...,...,...,...,...,...,...,...,...,...,...,...,...
345,CH3-bin.2,CH3-bin.0,4.612004e-07,0.000004,0.33,M_X,SC1_X,f_Clostridiaceae,Other,nucleotides and derivatives,Thymine C5H6N2O2,1.521961e-07
347,CH14-bin.1,CH14-bin.4,4.147959e-07,0.000206,0.18,CD_X,SC1_X,f_Lachnospiraceae,f_Desulfovibrionaceae,gases,Hydrogen,7.466326e-08
351,CH14-bin.4,CH14-bin.2,2.743474e-07,0.000008,0.73,CD_X,SC1_X,f_Desulfovibrionaceae,f_Lachnospiraceae,gases,Hydrogen sulfide,2.002736e-07
353,CH9-bin.2,CH9-bin.5,2.537980e-07,0.000007,0.22,CM_X,SC1_X,f_Clostridiaceae,f_Lactobacillaceae,gases,Hydrogen sulfide,5.583556e-08


### Overview of groups

In [302]:
pd.Series({family:len(mags) for family,mags in family_groups.items()})

Other                    19
f_Bacteroidaceae          4
f_CAG-74                  2
f_Clostridiaceae          3
f_Desulfobulbaceae        2
f_Desulfovibrionaceae    12
f_Dysgonomonadaceae       4
f_Fibrobacteraceae        2
f_Lachnospiraceae         8
f_Lactobacillaceae        2
f_Lentimicrobiaceae       2
f_Sphaerochaetaceae       6
f_Treponemataceae         4
f_Verruco-01              3
dtype: int64

In [303]:
all_mags_paper_reduced[all_mags_paper_reduced["new_coverage"]>10][["Source","Substrate","Family","Genus","new_coverage"]].sort_values(["Source","Substrate"])

,Source,Substrate,Family,Genus,new_coverage
MAG,,,,,
CH13-bin.12,Compost_Digestate,Avicel,f_Fibrobacteraceae,Fibro-01,36.526676
CH13-bin.11,Compost_Digestate,Avicel,f_Desulfovibrionaceae,Desulfovibrio,10.155790
CH13-bin.4,Compost_Digestate,Avicel,f_Bacteroidaceae,Bacteroides,10.043970
CH15-bin.0,Compost_Digestate,PASC,f_Treponemataceae,DUOS01,23.919040
CH15-bin.23,Compost_Digestate,PASC,Other,DTFZ01,10.319866
CH14-bin.1,Compost_Digestate,Xylan,f_Lachnospiraceae,Lacrimispora,54.707294
CH14-bin.2,Compost_Digestate,Xylan,f_Lachnospiraceae,Robinsoniella,18.831114
CH14-bin.0,Compost_Digestate,Xylan,Other,Caproicproducens,15.617491
CH8-bin.22,Cow_Manure,Avicel,f_Fibrobacteraceae,Fibro-01,42.725525


### Functions for statistics

In [304]:
def find_non_dependent(row,metric):
    return len(family_groups[row.name[0]]) - row[metric] #row.flux_mg


def statistics_adjustments(statistics_df):
    
    statistics_df = statistics_df.sort_values(by="p_value").copy()
    statistics_df["i"] = statistics_df["p_value"].rank(method="max")
    statistics_df["p_value_benjamini_h"] = statistics_df.apply(lambda row: min(row.p_value*statistics_df.shape[0]/row.i,1),axis=1)
    statistics_df.sort_index(inplace=True)
    return statistics_df


def statistics_function(steadier_sample_cross,dependent_variable,independent_variable,metric="flux_mg",metric_thresh=1e-6,pvalue_thresh=0.1):

    # Get average of each family according to each possible value of the independent variable
    # dependent_variable,dependent_variable.split("_")[1] here it decides if it is in the receiver or in the donor (dependent_variable.split("_")[1]) and groups by the [family_receiver,receiver,compound] and takes the mean of this
    steadiercom_crossfeeding_donor = steadier_sample_cross.loc[:,[dependent_variable,dependent_variable.split("_")[1],independent_variable,metric]].groupby([dependent_variable,dependent_variable.split("_")[1],independent_variable]).mean().copy()
    dependent = steadiercom_crossfeeding_donor[steadiercom_crossfeeding_donor[metric]>metric_thresh].reset_index().groupby([dependent_variable,independent_variable]).count().copy()
    not_dependent = dependent.apply(find_non_dependent,metric=metric,axis=1)

    # Add data for the missing values
    all_categories =set(not_dependent.index.get_level_values(1))

    for family in dependent.index.get_level_values(0):
        for category in all_categories-set(not_dependent.xs(family).index):
            not_dependent[(family,category)]=len(family_groups[family]) 
            
    concat_df = pd.concat({"dependent":dependent[metric],"not_dependent":not_dependent},axis=1).fillna(0)


    statistics = {}
    for independent_var in set(concat_df.index.get_level_values(1)):
        # Get the sub_df for the super class
        concat_df_sub = concat_df.xs(independent_var,level=1).copy()

        statistics[independent_var] = {}

        # For each row (each family)
        for i,row in concat_df_sub.iterrows():
            
            # Get the data for all other family
            other = concat_df_sub.loc[concat_df_sub.index[concat_df_sub.index!=i],:]
            
            data = pd.DataFrame({i:row,"other":other.sum()}).transpose().to_numpy()
            statistics[independent_var][(i,"data")]= data
            
            
            # Get odds ratio
            odds_ratio_num = data[0][0]
            odds_ratio_den = data[0][0] + data[0][1]
            other_num = data[1][0]
            other_den = data[1][0] + data[1][1]
            
            if odds_ratio_den == 0 or other_den == 0 or other_num==0:
                odds_ratio = math.inf
            else:
                odds_ratio = (odds_ratio_num / odds_ratio_den) / (other_num / other_den)
            statistics[independent_var][(i, "odds_ratio")] = odds_ratio
            
            
            # Calculate the Barnard exact statistical 
            p_value = stats.barnard_exact(pd.DataFrame({i:row,"other":other.sum()}).transpose().to_numpy(),alternative="greater")
            statistics[independent_var][(i,"p_value")]= p_value.pvalue
            

    statistics_df = pd.DataFrame(statistics)
    

    category_values = statistics_df.xs('p_value', level=1)
    

    values = {}
    for family,independent_var in category_values[category_values[category_values<pvalue_thresh].notnull()].stack().index:
        values[family,independent_var]= {"p_value":statistics_df.loc[(family,"p_value"),independent_var],"odds_ratio":statistics_df.loc[(family,"odds_ratio"),independent_var],"data":statistics_df.loc[(family,"data"),independent_var],"# interaction":concat_df.loc[(family,independent_var),"dependent"],"# no interaction":concat_df.loc[(family,independent_var),"not_dependent"]}

    significant = pd.DataFrame(values).transpose()
    if significant.empty==False:
        significant.index.names = (dependent_variable,independent_variable)

    return significant

## Likelyhood that family X is receiving compound A

In [305]:
statistics_df = statistics_function(steadier_sample_cross,"family_receiver","compound",metric="rate",metric_thresh=1e-6,pvalue_thresh=1)
statistics_df = statistics_adjustments(statistics_df)
statistics_df.shape

(205, 7)

In [306]:
statistics_df[(statistics_df.p_value_benjamini_h<0.05) & (statistics_df["odds_ratio"]>4.95)]

,,p_value,odds_ratio,data,# interaction,# no interaction,i,p_value_benjamini_h
family_receiver,compound,,,,,,,
f_Lactobacillaceae,L Ornithine C5H13N2O2,0.000104,inf,"[[2.0, 0.0], [0.0, 71.0]]",2.0,0,1.0,0.021409


In [307]:
statistics_df = statistics_function(steadier_sample_cross,"family_receiver","super_class",metric="rate",metric_thresh=1e-6,pvalue_thresh=1)
statistics_df = statistics_adjustments(statistics_df)
statistics_df.shape

(68, 7)

In [308]:
statistics_df[(statistics_df.p_value_benjamini_h<0.1) & (statistics_df["odds_ratio"]>4.95)]

,,p_value,odds_ratio,data,# interaction,# no interaction,i,p_value_benjamini_h
family_receiver,super_class,,,,,,,


## Likelyhood that family X is donating a compound A to another family

In [309]:
statistics_df = statistics_function(steadier_sample_cross,"family_donor","compound",metric="rate",metric_thresh=1e-6,pvalue_thresh=1)
statistics_df = statistics_adjustments(statistics_df)
statistics_df.shape

(172, 7)

In [310]:
statistics_df[(statistics_df.p_value_benjamini_h<0.05) & (statistics_df["odds_ratio"]>4.95)]

p_value odds_ratio  \
family_donor        compound                                     
f_Fibrobacteraceae  Indole                  0.00176      17.75   
f_Lachnospiraceae   Guanine                0.000903   6.770833   
f_Lactobacillaceae  L Ornithine C5H13N2O2  0.000104        inf   
f_Sphaerochaetaceae D-Glucose              0.000501   5.583333   
                    D-Mannose 1-phosphate  0.000197  14.888889   

                                                                 data  \
family_donor        compound                                            
f_Fibrobacteraceae  Indole                  [[2.0, 0.0], [4.0, 67.0]]   
f_Lachnospiraceae   Guanine                 [[5.0, 3.0], [6.0, 59.0]]   
f_Lactobacillaceae  L Ornithine C5H13N2O2   [[2.0, 0.0], [0.0, 71.0]]   
f_Sphaerochaetaceae D-Glucose              [[5.0, 1.0], [10.0, 57.0]]   
                    D-Mannose 1-phosphate   [[4.0, 2.0], [3.0, 64.0]]   

                                          # interaction # no interaction    i  \
family_donor        compound                                                    
f_Fibrobacteraceae  Indole                          2.0                0  7.0   
f_Lachnospiraceae   Guanine                         5.0                3  6.0   
f_Lactobacillaceae  L Ornithine C5H13N2O2           2.0                0  1.0   
f_Sphaerochaetaceae D-Glucose                       5.0                1  4.0   
                    D-Mannose 1-phosphate           4.0                2  2.0   

                                           p_value_benjamini_h  
family_donor        compound                                    
f_Fibrobacteraceae  Indole                            0.043238  
f_Lachnospiraceae   Guanine                           0.025897  
f_Lactobacillaceae  L Ornithine C5H13N2O2             0.017962  
f_Sphaerochaetaceae D-Glucose                         0.021544  
                    D-Mannose 1-phosphate             0.016979

In [311]:
statistics_df = statistics_function(steadier_sample_cross,"family_donor","super_class",metric="rate",metric_thresh=1e-6,pvalue_thresh=1)
statistics_df = statistics_adjustments(statistics_df)
statistics_df.shape

(68, 7)

In [312]:
statistics_df[(statistics_df.p_value_benjamini_h<0.1) & (statistics_df["odds_ratio"]>4.95)]

,,p_value,odds_ratio,data,# interaction,# no interaction,i,p_value_benjamini_h
family_donor,super_class,,,,,,,
f_Bacteroidaceae,organophosphorus compounds,0.001574,8.625,"[[3.0, 1.0], [6.0, 63.0]]",3.0,1,2.0,0.053506
f_Sphaerochaetaceae,organophosphorus compounds,0.000589,8.933333,"[[4.0, 2.0], [5.0, 62.0]]",4.0,2,1.0,0.040082


## Likelyhood that family X is receiving from family Y

In [313]:
statistics_df = statistics_function(steadier_sample_cross,"family_receiver","family_donor",metric="rate",metric_thresh=1e-6,pvalue_thresh=1)
statistics_df = statistics_adjustments(statistics_df)
statistics_df.shape

(93, 7)

In [314]:
statistics_df[(statistics_df.p_value_benjamini_h<0.05) & (statistics_df["odds_ratio"]>4.95)]

,,p_value,odds_ratio,data,# interaction,# no interaction,i,p_value_benjamini_h
family_receiver,family_donor,,,,,,,


### Who donates the most?

In [315]:
total_members = steadier_sample_cross.groupby("family_donor").nunique()["donor"]

### All communities - Relative abundance (above 10 %)

In [316]:
def abundance_statistic(abundance_communities,receiver_or_donor="receiver"):
    
    
    table_abundance_rec_don_dict = {}
    table_not_abundant_rec_don_dict = {}

    for compound in abundance_communities[abundance_communities.receiver_abundance_10].compound.unique():
        if receiver_or_donor=="receiver":
            abund_rec_don = len(abundance_communities[(abundance_communities.receiver_abundance_10) & (abundance_communities.compound==compound)].receiver.unique())
            not_abund_rec_don = len(abundance_communities[(~abundance_communities.receiver_abundance_10) & (abundance_communities.compound==compound)].receiver.unique())
        elif receiver_or_donor=="donor":
            abund_rec_don = len(abundance_communities[(abundance_communities.donor_abundance_10) & (abundance_communities.compound==compound)].donor.unique())
            not_abund_rec_don = len(abundance_communities[(~abundance_communities.donor_abundance_10) & (abundance_communities.compound==compound)].donor.unique())
        
        
        abund_not_rec_don = len(high_abundance) - abund_rec_don
        not_abund_not_rec_don = len(low_abunance) - not_abund_rec_don


        table = [[abund_rec_don,not_abund_rec_don],[abund_not_rec_don,not_abund_not_rec_don]]
        table_non_abundant = [[not_abund_rec_don,abund_rec_don],[not_abund_not_rec_don,abund_not_rec_don]]

        try:
            odds_ratio = (abund_rec_don/(abund_rec_don+abund_not_rec_don))/(not_abund_rec_don/(not_abund_rec_don+not_abund_not_rec_don))
        except:
            odds_ratio = math.inf


        p_value = stats.barnard_exact(table,alternative="greater")

        table_abundance_rec_don_dict[compound] = {"table":table,"p_value":p_value.pvalue,"odds_ratio":odds_ratio}


        p_value_not = stats.barnard_exact(table_non_abundant,alternative="greater")

        table_not_abundant_rec_don_dict[compound]= {"table":table,"p_value":p_value_not.pvalue,"odds_ratio":odds_ratio}
        
    return table_abundance_rec_don_dict, table_not_abundant_rec_don_dict
        


In [317]:
def abundance_statistic_super_class(abundance_communities,receiver_or_donor="receiver"):
    
    
    table_abundance_rec_don_dict = {}
    table_not_abundant_rec_don_dict = {}

    for super_class in abundance_communities[abundance_communities.receiver_abundance_10].super_class.unique():
        if receiver_or_donor=="receiver":
            abund_rec_don = len(abundance_communities[(abundance_communities.receiver_abundance_10) & (abundance_communities.super_class==super_class)].receiver.unique())
            not_abund_rec_don = len(abundance_communities[(~abundance_communities.receiver_abundance_10) & (abundance_communities.super_class==super_class)].receiver.unique())
        elif receiver_or_donor=="donor":
            abund_rec_don = len(abundance_communities[(abundance_communities.donor_abundance_10) & (abundance_communities.super_class==super_class)].donor.unique())
            not_abund_rec_don = len(abundance_communities[(~abundance_communities.donor_abundance_10) & (abundance_communities.super_class==super_class)].donor.unique())
        
        
        abund_not_rec_don = len(high_abundance) - abund_rec_don
        not_abund_not_rec_don = len(low_abunance) - not_abund_rec_don


        table = [[abund_rec_don,not_abund_rec_don],[abund_not_rec_don,not_abund_not_rec_don]]
        table_non_abundant = [[not_abund_rec_don,abund_rec_don],[not_abund_not_rec_don,abund_not_rec_don]]

        try:
            odds_ratio = (abund_rec_don/(abund_rec_don+abund_not_rec_don))/(not_abund_rec_don/(not_abund_rec_don+not_abund_not_rec_don))
        except:
            odds_ratio = math.inf


        p_value = stats.barnard_exact(table,alternative="greater")

        table_abundance_rec_don_dict[super_class] = {"table":table,"p_value":p_value.pvalue,"odds_ratio":odds_ratio}


        p_value_not = stats.barnard_exact(table_non_abundant,alternative="greater")

        table_not_abundant_rec_don_dict[super_class]= {"table":table,"p_value":p_value_not.pvalue,"odds_ratio":odds_ratio}
        
    return table_abundance_rec_don_dict, table_not_abundant_rec_don_dict
        


**Assign abundant/not-abundant classifier**

In [318]:
abundance_communities = steadier_sample_cross.copy()
abundance_communities["donor_abundance_10"] = abundance_communities["donor"].map(lambda x: all_mags_paper.loc[x,"new_coverage"]>10)
abundance_communities["receiver_abundance_10"] = abundance_communities["receiver"].map(lambda x: all_mags_paper.loc[x,"new_coverage"]>10)

**Overview of abundant members**

In [319]:
high_abundance = set(list(abundance_communities[abundance_communities.donor_abundance_10].donor.unique()) + list(abundance_communities[abundance_communities.receiver_abundance_10].receiver.unique()))
low_abunance = set(list(abundance_communities[~abundance_communities.donor_abundance_10].donor.unique()) + list(abundance_communities[~abundance_communities.receiver_abundance_10].receiver.unique()))

#### Statistics for receiving

In [320]:
table_abundance_rec_dict, table_not_abundant_rec_dict = abundance_statistic(abundance_communities,receiver_or_donor="receiver")

**Abundant -compounds**

In [321]:
abund_rec_df = pd.DataFrame(table_abundance_rec_dict).T
abund_rec_df = statistics_adjustments(abund_rec_df)

abund_rec_df[(abund_rec_df.p_value_benjamini_h<0.05)&(abund_rec_df.odds_ratio>4.95)].sort_values("p_value")

,table,p_value,odds_ratio,i,p_value_benjamini_h


**Not abundant -compounds**

In [322]:
not_abund_rec_df = pd.DataFrame(table_not_abundant_rec_dict).T
not_abund_rec_df = statistics_adjustments(not_abund_rec_df)
not_abund_rec_df["odds_ratio"] = not_abund_rec_df["odds_ratio"].map(lambda x: 1/x)
not_abund_rec_df[(not_abund_rec_df.p_value_benjamini_h<0.05)& (not_abund_rec_df.odds_ratio>4.95)].sort_values("p_value")

,table,p_value,odds_ratio,i,p_value_benjamini_h


**SUPER CLASS**

In [323]:
table_abundance_rec_dict, table_not_abundant_rec_dict = abundance_statistic_super_class(abundance_communities,receiver_or_donor="receiver")

**Not abundant -superclass**

In [324]:
abund_rec_df = pd.DataFrame(table_abundance_rec_dict).T
abund_rec_df = statistics_adjustments(abund_rec_df)

abund_rec_df[(abund_rec_df.p_value_benjamini_h<0.05)& (abund_rec_df
                                                       .odds_ratio>4.95)].sort_values("p_value")

,table,p_value,odds_ratio,i,p_value_benjamini_h


**Abundant -compounds**

In [325]:
not_abund_rec_df = pd.DataFrame(table_not_abundant_rec_dict).T
not_abund_rec_df = statistics_adjustments(not_abund_rec_df)
not_abund_rec_df["odds_ratio"] = not_abund_rec_df["odds_ratio"].map(lambda x: 1/x)
not_abund_rec_df[(not_abund_rec_df.p_value_benjamini_h<0.05)& (not_abund_rec_df.odds_ratio>4.95)].sort_values("p_value")

,table,p_value,odds_ratio,i,p_value_benjamini_h


#### Statistics for donating

In [326]:
table_abundant_donation_dict, table_not_abundant_donation_dict = abundance_statistic(abundance_communities,receiver_or_donor="donor")

**Abundant -compound**

In [327]:
abundance_don_df = pd.DataFrame(table_abundant_donation_dict).T
abundance_don_df = statistics_adjustments(abundance_don_df)
abundance_don_df[(abundance_don_df.p_value_benjamini_h<0.05) & (abundance_don_df.odds_ratio>4.95)].sort_values("p_value")


,table,p_value,odds_ratio,i,p_value_benjamini_h
Uracil,"[[8, 4], [7, 54]]",0.000083,7.733333,1.0,0.002904
Thymine C5H6N2O2,"[[7, 3], [8, 55]]",0.000123,9.022222,2.0,0.002149
Indole,"[[5, 1], [10, 57]]",0.000501,19.333333,3.0,0.005845
D-Glyceraldehyde,"[[4, 2], [11, 56]]",0.005843,7.733333,7.0,0.029216


**Not abundant -compound**

In [328]:
not_abundant_don_df = pd.DataFrame(table_not_abundant_donation_dict).T
not_abundant_don_df = statistics_adjustments(not_abundant_don_df)


not_abundant_don_df["odds_ratio"] = not_abundant_don_df["odds_ratio"].map(lambda x: math.inf if x==0 else 1/x)
not_abundant_don_df[(not_abundant_don_df.p_value_benjamini_h<0.05)& (not_abundant_don_df.odds_ratio>4.95)].sort_values("p_value")

,table,p_value,odds_ratio,i,p_value_benjamini_h


**SUPER CLASS**

In [329]:
table_abundant_donation_dict, table_not_abundant_donation_dict = abundance_statistic_super_class(abundance_communities,receiver_or_donor="donor")

**Abundant**

In [330]:
abundance_don_df = pd.DataFrame(table_abundant_donation_dict).T
abundance_don_df = statistics_adjustments(abundance_don_df)
abundance_don_df[(abundance_don_df.p_value_benjamini_h<0.05)& (abundance_don_df.odds_ratio>4.95)].sort_values("p_value")


,table,p_value,odds_ratio,i,p_value_benjamini_h


**Not abundant**

In [331]:
not_abundant_don_df = pd.DataFrame(table_not_abundant_donation_dict).T
not_abundant_don_df = statistics_adjustments(not_abundant_don_df)


not_abundant_don_df["odds_ratio"] = not_abundant_don_df["odds_ratio"].map(lambda x: math.inf if x==0 else 1/x)
not_abundant_don_df[(not_abundant_don_df.p_value_benjamini_h<0.05) & (not_abundant_don_df.odds_ratio>4.95)].sort_values("p_value")

,table,p_value,odds_ratio,i,p_value_benjamini_h


## Mass flow

#### Xylose

**Receiver of Xylose**

In [332]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Xylose")].value_counts("family_receiver")

family_receiver
Other                    4
f_Desulfovibrionaceae    2
f_Lachnospiraceae        2
f_Lactobacillaceae       2
f_Clostridiaceae         1
dtype: int64

In [333]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Xylose")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
Other                    0.054139
f_Clostridiaceae         0.009273
f_Lachnospiraceae        0.004283
f_Lactobacillaceae       0.002597
f_Desulfovibrionaceae    0.000531
Name: mass_rate*frequency, dtype: float64

**Donor of Xylose**

In [334]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Xylose")].value_counts("family_donor")

family_donor
f_Lachnospiraceae    5
Other                4
f_Clostridiaceae     2
dtype: int64

In [335]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Xylose")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
Other                0.054645
f_Clostridiaceae     0.009479
f_Lachnospiraceae    0.006698
Name: mass_rate*frequency, dtype: float64

In [336]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Xylose")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver      
CD_X       Other                    0.004780
           f_Lachnospiraceae        0.001387
           f_Desulfovibrionaceae    0.000531
CM_X       Other                    0.049152
           f_Lachnospiraceae        0.002896
           f_Lactobacillaceae       0.002597
M_X        f_Clostridiaceae         0.009273
           Other                    0.000207
Name: mass_rate*frequency, dtype: float64

In [337]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Xylose")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor     
CD_X       f_Lachnospiraceae    0.006698
CM_X       Other                0.054645
M_X        f_Clostridiaceae     0.009479
Name: mass_rate*frequency, dtype: float64

#### Acetaldehyde

**Receiver of Acetaldehyde**

In [338]:
steadier_sample_cross[(steadier_sample_cross.compound=="Acetaldehyde")].value_counts("family_receiver")

family_receiver
Other                    32
f_Desulfovibrionaceae    25
f_Sphaerochaetaceae      23
f_Treponemataceae        10
f_Lachnospiraceae         9
f_CAG-74                  8
f_Desulfobulbaceae        8
f_Verruco-01              6
f_Lactobacillaceae        4
f_Clostridiaceae          2
f_Dysgonomonadaceae       2
dtype: int64

In [339]:
steadier_sample_cross[(steadier_sample_cross.compound=="Acetaldehyde")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
Other                    0.009970
f_Desulfovibrionaceae    0.006450
f_Sphaerochaetaceae      0.005201
f_Lactobacillaceae       0.003790
f_Lachnospiraceae        0.003349
f_Verruco-01             0.002947
f_Treponemataceae        0.001093
f_Clostridiaceae         0.001012
f_Desulfobulbaceae       0.000695
f_CAG-74                 0.000263
f_Dysgonomonadaceae      0.000002
Name: mass_rate*frequency, dtype: float64

**Donor of Acetaldehyde**

In [340]:
steadier_sample_cross[(steadier_sample_cross.compound=="Acetaldehyde")].value_counts("family_donor")

family_donor
Other                    40
f_Lachnospiraceae        20
f_Verruco-01             13
f_Desulfovibrionaceae    11
f_Dysgonomonadaceae      10
f_Treponemataceae         9
f_Clostridiaceae          8
f_Fibrobacteraceae        8
f_Lentimicrobiaceae       6
f_Sphaerochaetaceae       4
dtype: int64

In [341]:
steadier_sample_cross[(steadier_sample_cross.compound=="Acetaldehyde")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
Other                    0.012051
f_Lachnospiraceae        0.005636
f_Desulfovibrionaceae    0.004056
f_Clostridiaceae         0.004054
f_Treponemataceae        0.003374
f_Verruco-01             0.002553
f_Fibrobacteraceae       0.002356
f_Dysgonomonadaceae      0.000406
f_Lentimicrobiaceae      0.000223
f_Sphaerochaetaceae      0.000065
Name: mass_rate*frequency, dtype: float64

In [342]:
steadier_sample_cross[(steadier_sample_cross.compound=="Acetaldehyde")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver      
CD_A       f_Sphaerochaetaceae      0.000995
           f_Desulfovibrionaceae    0.000768
           f_Lachnospiraceae        0.000514
           f_Treponemataceae        0.000288
           Other                    0.000014
CD_P       Other                    0.002212
           f_Desulfovibrionaceae    0.001372
           f_Sphaerochaetaceae      0.000887
           f_Lachnospiraceae        0.000178
           f_Desulfobulbaceae       0.000164
           f_Dysgonomonadaceae      0.000002
CD_X       f_Lachnospiraceae        0.001147
           f_Desulfovibrionaceae    0.000530
CM_A       f_Verruco-01             0.002947
           f_Treponemataceae        0.000805
           f_Sphaerochaetaceae      0.000570
           f_CAG-74                 0.000156
           Other                    0.000036
CM_P       f_Sphaerochaetaceae      0.002748
           f_Desulfovibrionaceae    0.001479
           f_Desulfobulbaceae       0.000531
           Other      

In [343]:
steadier_sample_cross[(steadier_sample_cross.compound=="Acetaldehyde")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor         
CD_A       f_Fibrobacteraceae       2.355715e-03
           f_Lachnospiraceae        2.229468e-04
           f_Desulfovibrionaceae    8.351066e-07
CD_P       f_Treponemataceae        3.373540e-03
           Other                    8.050396e-04
           f_Dysgonomonadaceae      3.848677e-04
           f_Lentimicrobiaceae      2.234827e-04
           f_Lachnospiraceae        2.783310e-05
CD_X       f_Lachnospiraceae        1.676431e-03
CM_A       Other                    2.943162e-03
           f_Verruco-01             1.535072e-03
           f_Dysgonomonadaceae      2.159235e-05
           f_Sphaerochaetaceae      1.520082e-05
CM_P       f_Desulfovibrionaceae    3.942029e-03
           f_Verruco-01             1.017542e-03
           Other                    1.018360e-04
           f_Sphaerochaetaceae      4.957513e-05
CM_X       Other                    8.200893e-03
           f_Clostridiaceae         2.889000e-03
M_P        f_Lachnospiraceae        

#### Glucose

**Receiver of Glucose**

In [344]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Glucose")].value_counts("family_receiver")

family_receiver
f_Desulfovibrionaceae    10
Other                     7
f_Sphaerochaetaceae       7
f_Verruco-01              4
f_Lentimicrobiaceae       3
f_Desulfobulbaceae        1
dtype: int64

In [345]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Glucose")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
f_Desulfovibrionaceae    0.006913
f_Sphaerochaetaceae      0.005009
f_Verruco-01             0.001583
Other                    0.001222
f_Lentimicrobiaceae      0.001204
f_Desulfobulbaceae       0.000130
Name: mass_rate*frequency, dtype: float64

**Donor of D-Glucose**

In [346]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Glucose")].value_counts("family_donor")

family_donor
f_Sphaerochaetaceae      13
Other                     6
f_Treponemataceae         5
f_Verruco-01              4
f_Lachnospiraceae         3
f_Desulfovibrionaceae     1
dtype: int64

In [347]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Glucose")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
f_Sphaerochaetaceae      0.006929
f_Treponemataceae        0.003833
Other                    0.002237
f_Verruco-01             0.002197
f_Lachnospiraceae        0.000681
f_Desulfovibrionaceae    0.000185
Name: mass_rate*frequency, dtype: float64

In [348]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Glucose")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver      
CD_A       f_Sphaerochaetaceae      0.000541
           f_Desulfovibrionaceae    0.000250
CD_P       f_Desulfovibrionaceae    0.002872
           f_Sphaerochaetaceae      0.001329
           f_Lentimicrobiaceae      0.001204
CM_A       f_Verruco-01             0.001583
           f_Sphaerochaetaceae      0.001527
CM_P       f_Desulfovibrionaceae    0.003791
           f_Sphaerochaetaceae      0.001612
           Other                    0.001183
           f_Desulfobulbaceae       0.000130
M_P        Other                    0.000039
Name: mass_rate*frequency, dtype: float64

In [349]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Glucose")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor         
CD_A       f_Lachnospiraceae        0.000642
           f_Sphaerochaetaceae      0.000103
           f_Treponemataceae        0.000047
CD_P       f_Treponemataceae        0.003787
           Other                    0.000924
           f_Sphaerochaetaceae      0.000695
CM_A       f_Verruco-01             0.001396
           Other                    0.001225
           f_Sphaerochaetaceae      0.000304
           f_Desulfovibrionaceae    0.000185
CM_P       f_Sphaerochaetaceae      0.005827
           f_Verruco-01             0.000801
           Other                    0.000088
M_P        f_Lachnospiraceae        0.000039
Name: mass_rate*frequency, dtype: float64

#### Ribose

**Receiver of Ribose**

In [350]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Ribose")].value_counts("family_receiver")

family_receiver
Other                    6
f_Bacteroidaceae         1
f_Clostridiaceae         1
f_Desulfobulbaceae       1
f_Desulfovibrionaceae    1
f_Lachnospiraceae        1
f_Treponemataceae        1
dtype: int64

In [351]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Ribose")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
Other                    0.014869
f_Bacteroidaceae         0.001610
f_Desulfobulbaceae       0.000120
f_Desulfovibrionaceae    0.000108
f_Clostridiaceae         0.000072
f_Lachnospiraceae        0.000055
f_Treponemataceae        0.000007
Name: mass_rate*frequency, dtype: float64

####

**Donor of D-Ribose**

In [352]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Ribose")].value_counts("family_donor")

family_donor
f_Lachnospiraceae      4
Other                  3
f_Sphaerochaetaceae    2
f_Treponemataceae      2
f_Bacteroidaceae       1
dtype: int64

In [353]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Ribose")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
Other                  0.014897
f_Sphaerochaetaceae    0.001629
f_Lachnospiraceae      0.000181
f_Treponemataceae      0.000124
f_Bacteroidaceae       0.000011
Name: mass_rate*frequency, dtype: float64

In [354]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Ribose")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver      
CD_A       Other                    0.000037
           f_Treponemataceae        0.000007
CD_P       f_Desulfobulbaceae       0.000120
           Other                    0.000004
CD_X       f_Desulfovibrionaceae    0.000108
CM_A       Other                    0.000019
CM_P       f_Bacteroidaceae         0.001610
CM_X       Other                    0.014770
           f_Clostridiaceae         0.000072
           f_Lachnospiraceae        0.000055
M_P        Other                    0.000039
Name: mass_rate*frequency, dtype: float64

In [355]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Ribose")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor       
CD_A       f_Lachnospiraceae      0.000034
           f_Bacteroidaceae       0.000011
CD_P       f_Treponemataceae      0.000124
CD_X       f_Lachnospiraceae      0.000108
CM_A       f_Sphaerochaetaceae    0.000019
CM_P       f_Sphaerochaetaceae    0.001610
CM_X       Other                  0.014897
M_P        f_Lachnospiraceae      0.000039
Name: mass_rate*frequency, dtype: float64

#### L-Lactate

**Receiver of L-Lactate**

In [356]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lactate")].value_counts("family_receiver")

family_receiver
f_Desulfovibrionaceae    14
Other                     5
f_Lachnospiraceae         4
f_Desulfobulbaceae        3
f_Clostridiaceae          1
dtype: int64

In [357]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lactate")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
f_Lachnospiraceae        0.003146
f_Clostridiaceae         0.001763
f_Desulfovibrionaceae    0.001630
Other                    0.000230
f_Desulfobulbaceae       0.000154
Name: mass_rate*frequency, dtype: float64

**Donor of L-Lactate**

In [358]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lactate")].value_counts("family_donor")

family_donor
f_Desulfovibrionaceae    13
Other                     5
f_Desulfobulbaceae        3
f_Lachnospiraceae         3
f_Dysgonomonadaceae       2
f_Clostridiaceae          1
dtype: int64

In [359]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lactate")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
f_Desulfovibrionaceae    4.766334e-03
f_Clostridiaceae         1.762658e-03
f_Lachnospiraceae        1.632385e-04
Other                    1.400399e-04
f_Desulfobulbaceae       9.009249e-05
f_Dysgonomonadaceae      3.597642e-07
Name: mass_rate*frequency, dtype: float64

In [360]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lactate")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver      
CD_A       f_Lachnospiraceae        0.001130
           Other                    0.000082
           f_Desulfovibrionaceae    0.000027
CD_P       f_Desulfovibrionaceae    0.000264
CD_X       f_Desulfovibrionaceae    0.000020
           Other                    0.000019
CM_A       f_Desulfovibrionaceae    0.000570
CM_P       f_Desulfovibrionaceae    0.000749
           f_Desulfobulbaceae       0.000154
M_P        f_Lachnospiraceae        0.002016
           Other                    0.000128
M_X        f_Clostridiaceae         0.001763
Name: mass_rate*frequency, dtype: float64

In [361]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lactate")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor         
CD_A       f_Desulfovibrionaceae    1.205437e-03
           Other                    3.345515e-05
CD_P       f_Lachnospiraceae        1.429130e-04
           f_Desulfovibrionaceae    6.546571e-05
           f_Desulfobulbaceae       5.563967e-05
           f_Dysgonomonadaceae      3.597642e-07
CD_X       Other                    1.995621e-05
           f_Desulfovibrionaceae    1.937593e-05
CM_A       f_Desulfovibrionaceae    5.703976e-04
CM_P       f_Desulfovibrionaceae    8.686773e-04
           f_Desulfobulbaceae       3.445282e-05
M_P        f_Desulfovibrionaceae    2.036981e-03
           Other                    8.662856e-05
           f_Lachnospiraceae        2.032548e-05
M_X        f_Clostridiaceae         1.762658e-03
Name: mass_rate*frequency, dtype: float64

#### L-Malate

**Receiver of L-Malate**

In [362]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Malate")].value_counts("family_receiver")

family_receiver
Other                    10
f_Desulfovibrionaceae    10
f_Clostridiaceae          2
f_Lachnospiraceae         2
f_Lactobacillaceae        2
f_Treponemataceae         2
f_Bacteroidaceae          1
f_Desulfobulbaceae        1
f_Dysgonomonadaceae       1
dtype: int64

In [363]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Malate")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
f_Clostridiaceae         0.003305
f_Lactobacillaceae       0.000476
f_Treponemataceae        0.000332
Other                    0.000305
f_Desulfovibrionaceae    0.000195
f_Lachnospiraceae        0.000149
f_Dysgonomonadaceae      0.000059
f_Desulfobulbaceae       0.000043
f_Bacteroidaceae         0.000004
Name: mass_rate*frequency, dtype: float64

**Donor of L-Malate**

In [364]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Malate")].value_counts("family_donor")

family_donor
f_Lachnospiraceae      10
f_Sphaerochaetaceae     6
f_Treponemataceae       6
Other                   4
f_Bacteroidaceae        2
f_Clostridiaceae        2
f_CAG-74                1
dtype: int64

In [365]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Malate")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
Other                  0.003574
f_Lachnospiraceae      0.000615
f_Treponemataceae      0.000484
f_Sphaerochaetaceae    0.000115
f_Clostridiaceae       0.000070
f_CAG-74               0.000008
f_Bacteroidaceae       0.000002
Name: mass_rate*frequency, dtype: float64

In [366]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Malate")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver      
CD_A       f_Treponemataceae        3.318087e-04
           f_Desulfovibrionaceae    7.291247e-05
           Other                    3.136580e-05
CD_P       Other                    1.751037e-04
           f_Lachnospiraceae        1.391037e-04
           f_Dysgonomonadaceae      5.876896e-05
           f_Desulfovibrionaceae    4.511447e-05
           f_Desulfobulbaceae       4.265540e-05
CD_X       f_Desulfovibrionaceae    4.471963e-05
CM_A       f_Desulfovibrionaceae    3.256037e-05
CM_P       f_Bacteroidaceae         3.854112e-06
           Other                    2.835532e-06
           f_Desulfovibrionaceae    7.537451e-08
CM_X       f_Clostridiaceae         3.305331e-03
           f_Lactobacillaceae       4.756640e-04
M_P        Other                    2.602933e-05
           f_Lachnospiraceae        1.019501e-05
M_X        Other                    6.978280e-05
Name: mass_rate*frequency, dtype: float64

In [367]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Malate")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor       
CD_A       f_Lachnospiraceae      3.260580e-04
           f_Sphaerochaetaceae    1.100289e-04
CD_P       f_Treponemataceae      4.607462e-04
CD_X       f_Lachnospiraceae      4.471963e-05
CM_A       f_Treponemataceae      2.355572e-05
           f_CAG-74               8.206340e-06
           Other                  7.983096e-07
CM_P       f_Sphaerochaetaceae    5.162206e-06
           f_Bacteroidaceae       1.602812e-06
CM_X       Other                  3.569956e-03
           f_Lachnospiraceae      2.110390e-04
M_P        f_Lachnospiraceae      3.335311e-05
           Other                  2.871238e-06
M_X        f_Clostridiaceae       6.978280e-05
Name: mass_rate*frequency, dtype: float64

#### D-Aspartate

**Receiver of D-Aspartate**

In [368]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Aspartate")].value_counts("family_receiver")

family_receiver
f_Lachnospiraceae        2
Other                    1
f_Clostridiaceae         1
f_Desulfovibrionaceae    1
f_Lentimicrobiaceae      1
dtype: int64

In [369]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Aspartate")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
f_Clostridiaceae         0.002110
f_Lachnospiraceae        0.000718
f_Desulfovibrionaceae    0.000130
Other                    0.000013
f_Lentimicrobiaceae      0.000002
Name: mass_rate*frequency, dtype: float64

**Donor of D-Aspartate**

In [370]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Aspartate")].value_counts("family_donor")

family_donor
f_Lachnospiraceae      3
Other                  1
f_Clostridiaceae       1
f_Lentimicrobiaceae    1
dtype: int64

In [371]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Aspartate")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
f_Clostridiaceae       0.002110
f_Lachnospiraceae      0.000848
f_Lentimicrobiaceae    0.000013
Other                  0.000002
Name: mass_rate*frequency, dtype: float64

In [372]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Aspartate")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver      
CD_X       f_Lachnospiraceae        0.000605
           f_Desulfovibrionaceae    0.000130
CM_P       Other                    0.000013
           f_Lentimicrobiaceae      0.000002
M_P        f_Lachnospiraceae        0.000113
M_X        f_Clostridiaceae         0.002110
Name: mass_rate*frequency, dtype: float64

In [373]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Aspartate")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor       
CD_X       f_Lachnospiraceae      0.000735
CM_P       f_Lentimicrobiaceae    0.000013
           Other                  0.000002
M_P        f_Lachnospiraceae      0.000113
M_X        f_Clostridiaceae       0.002110
Name: mass_rate*frequency, dtype: float64

#### L-Aspartate

**Receiver of L-Aspartate**

In [374]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Aspartate")].value_counts("family_receiver")

family_receiver
f_Fibrobacteraceae       2
f_Sphaerochaetaceae      2
f_Treponemataceae        2
f_CAG-74                 1
f_Desulfovibrionaceae    1
f_Dysgonomonadaceae      1
f_Lachnospiraceae        1
f_Lentimicrobiaceae      1
f_Verruco-01             1
dtype: int64

In [375]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Aspartate")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
f_Fibrobacteraceae       0.001878
f_Treponemataceae        0.000064
f_Sphaerochaetaceae      0.000034
f_Desulfovibrionaceae    0.000033
f_Dysgonomonadaceae      0.000024
f_Lachnospiraceae        0.000015
f_Verruco-01             0.000014
f_CAG-74                 0.000007
f_Lentimicrobiaceae      0.000005
Name: mass_rate*frequency, dtype: float64

**Donor of L-Aspartate**

In [376]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Aspartate")].value_counts("family_donor")

family_donor
f_Desulfovibrionaceae    7
f_Lachnospiraceae        5
dtype: int64

In [377]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Aspartate")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
f_Desulfovibrionaceae    0.001705
f_Lachnospiraceae        0.000367
Name: mass_rate*frequency, dtype: float64

In [378]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Aspartate")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver      
CD_A       f_Fibrobacteraceae       0.000255
           f_Treponemataceae        0.000064
           f_Desulfovibrionaceae    0.000033
           f_Lachnospiraceae        0.000015
CM_A       f_Fibrobacteraceae       0.001623
CM_P       f_Sphaerochaetaceae      0.000034
           f_Dysgonomonadaceae      0.000024
           f_Verruco-01             0.000014
           f_CAG-74                 0.000007
           f_Lentimicrobiaceae      0.000005
Name: mass_rate*frequency, dtype: float64

In [379]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Aspartate")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor         
CD_A       f_Lachnospiraceae        0.000367
CM_A       f_Desulfovibrionaceae    0.001623
CM_P       f_Desulfovibrionaceae    0.000082
Name: mass_rate*frequency, dtype: float64

#### Guanine

**Receiver of Guanine**

In [380]:
steadier_sample_cross[(steadier_sample_cross.compound=="Guanine")].value_counts("family_receiver")

family_receiver
Other                    26
f_Desulfovibrionaceae    15
f_Lachnospiraceae         7
f_Sphaerochaetaceae       7
f_Dysgonomonadaceae       6
f_Treponemataceae         5
f_Bacteroidaceae          4
f_Lentimicrobiaceae       3
f_Verruco-01              3
f_CAG-74                  2
f_Clostridiaceae          2
f_Fibrobacteraceae        2
f_Lactobacillaceae        2
f_Desulfobulbaceae        1
dtype: int64

In [381]:
steadier_sample_cross[(steadier_sample_cross.compound=="Guanine")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
Other                    0.000714
f_Desulfovibrionaceae    0.000236
f_Lachnospiraceae        0.000109
f_Sphaerochaetaceae      0.000088
f_Clostridiaceae         0.000086
f_Bacteroidaceae         0.000062
f_Verruco-01             0.000035
f_Fibrobacteraceae       0.000034
f_Treponemataceae        0.000032
f_Lactobacillaceae       0.000027
f_Dysgonomonadaceae      0.000026
f_CAG-74                 0.000019
f_Lentimicrobiaceae      0.000016
f_Desulfobulbaceae       0.000013
Name: mass_rate*frequency, dtype: float64

**Donor of Guanine**

In [382]:
steadier_sample_cross[(steadier_sample_cross.compound=="Guanine")].value_counts("family_donor")

family_donor
f_Lachnospiraceae        25
f_Desulfovibrionaceae    23
f_Bacteroidaceae         12
f_Treponemataceae        12
f_Sphaerochaetaceae      11
f_Clostridiaceae          2
dtype: int64

In [383]:
steadier_sample_cross[(steadier_sample_cross.compound=="Guanine")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
f_Lachnospiraceae        0.000732
f_Desulfovibrionaceae    0.000314
f_Treponemataceae        0.000224
f_Sphaerochaetaceae      0.000110
f_Clostridiaceae         0.000077
f_Bacteroidaceae         0.000040
Name: mass_rate*frequency, dtype: float64

In [384]:
steadier_sample_cross[(steadier_sample_cross.compound=="Guanine")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver      
CD_A       f_Fibrobacteraceae       0.000034
           f_Bacteroidaceae         0.000031
           f_Sphaerochaetaceae      0.000025
           f_Treponemataceae        0.000022
           f_Lachnospiraceae        0.000021
           Other                    0.000016
           f_Desulfovibrionaceae    0.000005
CD_P       Other                    0.000158
           f_Desulfovibrionaceae    0.000053
           f_Dysgonomonadaceae      0.000018
           f_Sphaerochaetaceae      0.000016
           f_Lentimicrobiaceae      0.000011
           f_Lachnospiraceae        0.000008
CD_X       f_Lachnospiraceae        0.000067
           Other                    0.000055
           f_Desulfovibrionaceae    0.000010
CM_A       Other                    0.000045
           f_Verruco-01             0.000020
           f_Desulfovibrionaceae    0.000019
           f_Treponemataceae        0.000010
           f_CAG-74                 0.000007
           f_Bacteroid

In [385]:
steadier_sample_cross[(steadier_sample_cross.compound=="Guanine")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor         
CD_A       f_Desulfovibrionaceae    1.076909e-04
           f_Lachnospiraceae        4.626516e-05
CD_P       f_Treponemataceae        2.239884e-04
           f_Bacteroidaceae         3.971050e-05
           f_Lachnospiraceae        4.633073e-07
CD_X       f_Lachnospiraceae        1.319849e-04
CM_A       f_Sphaerochaetaceae      1.098016e-04
CM_P       f_Desulfovibrionaceae    2.061102e-04
CM_X       f_Lachnospiraceae        3.326406e-04
M_P        f_Lachnospiraceae        2.206873e-04
M_X        f_Clostridiaceae         7.728629e-05
Name: mass_rate*frequency, dtype: float64

#### Uracil

**Receiver of Uracil**

In [386]:
steadier_sample_cross[(steadier_sample_cross.compound=="Uracil")].value_counts("family_receiver")

family_receiver
Other                    24
f_Desulfovibrionaceae    17
f_Lachnospiraceae        11
f_Bacteroidaceae          7
f_Dysgonomonadaceae       6
f_Treponemataceae         6
f_Sphaerochaetaceae       5
f_Fibrobacteraceae        4
f_Desulfobulbaceae        3
f_Lentimicrobiaceae       3
f_CAG-74                  2
f_Clostridiaceae          2
f_Lactobacillaceae        2
dtype: int64

In [387]:
steadier_sample_cross[(steadier_sample_cross.compound=="Uracil")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
Other                    0.000362
f_Lachnospiraceae        0.000306
f_Fibrobacteraceae       0.000300
f_Desulfovibrionaceae    0.000130
f_Bacteroidaceae         0.000085
f_Clostridiaceae         0.000080
f_Sphaerochaetaceae      0.000067
f_Treponemataceae        0.000033
f_Dysgonomonadaceae      0.000023
f_Desulfobulbaceae       0.000019
f_Lentimicrobiaceae      0.000016
f_Lactobacillaceae       0.000012
f_CAG-74                 0.000010
Name: mass_rate*frequency, dtype: float64

**Donor of Uracil**

In [388]:
steadier_sample_cross[(steadier_sample_cross.compound=="Uracil")].value_counts("family_donor")

family_donor
f_Desulfovibrionaceae    27
Other                    23
f_Sphaerochaetaceae      18
f_Treponemataceae        14
f_Lachnospiraceae         8
f_Clostridiaceae          2
dtype: int64

In [389]:
steadier_sample_cross[(steadier_sample_cross.compound=="Uracil")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
f_Desulfovibrionaceae    0.000445
f_Sphaerochaetaceae      0.000435
f_Lachnospiraceae        0.000226
f_Treponemataceae        0.000191
f_Clostridiaceae         0.000078
Other                    0.000069
Name: mass_rate*frequency, dtype: float64

In [390]:
steadier_sample_cross[(steadier_sample_cross.compound=="Uracil")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver      
CD_A       f_Fibrobacteraceae       0.000135
           f_Lachnospiraceae        0.000036
           f_Treponemataceae        0.000022
           f_Bacteroidaceae         0.000022
           f_Desulfovibrionaceae    0.000019
           Other                    0.000015
CD_P       Other                    0.000068
           f_Desulfovibrionaceae    0.000048
           f_Bacteroidaceae         0.000030
           f_Sphaerochaetaceae      0.000020
           f_Dysgonomonadaceae      0.000017
           f_Lachnospiraceae        0.000014
           f_Lentimicrobiaceae      0.000012
           f_Desulfobulbaceae       0.000006
CD_X       f_Lachnospiraceae        0.000090
           Other                    0.000024
           f_Desulfovibrionaceae    0.000010
CM_A       f_Fibrobacteraceae       0.000165
           Other                    0.000035
           f_Desulfovibrionaceae    0.000022
           f_Treponemataceae        0.000011
           f_Bacteroid

In [391]:
steadier_sample_cross[(steadier_sample_cross.compound=="Uracil")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor         
CD_A       f_Sphaerochaetaceae      0.000194
           f_Desulfovibrionaceae    0.000054
CD_P       f_Treponemataceae        0.000191
           Other                    0.000023
CD_X       f_Lachnospiraceae        0.000082
           Other                    0.000041
CM_A       f_Sphaerochaetaceae      0.000240
           Other                    0.000005
CM_P       f_Desulfovibrionaceae    0.000160
CM_X       f_Lachnospiraceae        0.000144
M_P        f_Desulfovibrionaceae    0.000231
M_X        f_Clostridiaceae         0.000078
Name: mass_rate*frequency, dtype: float64

#### L-Arginine

**Receiver of L-Arginine**

In [392]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Arginine")].value_counts("family_receiver")

family_receiver
Other                    16
f_Desulfovibrionaceae     7
f_Dysgonomonadaceae       5
f_Sphaerochaetaceae       5
f_Bacteroidaceae          4
f_CAG-74                  4
f_Treponemataceae         3
f_Verruco-01              3
f_Clostridiaceae          2
f_Desulfobulbaceae        2
f_Lachnospiraceae         2
f_Lactobacillaceae        2
f_Lentimicrobiaceae       2
f_Fibrobacteraceae        1
dtype: int64

In [393]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Arginine")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
Other                    0.000551
f_Clostridiaceae         0.000348
f_Fibrobacteraceae       0.000099
f_Bacteroidaceae         0.000087
f_Treponemataceae        0.000071
f_Desulfovibrionaceae    0.000053
f_Sphaerochaetaceae      0.000052
f_Verruco-01             0.000038
f_Dysgonomonadaceae      0.000029
f_Lentimicrobiaceae      0.000025
f_Lactobacillaceae       0.000021
f_Desulfobulbaceae       0.000018
f_CAG-74                 0.000017
f_Lachnospiraceae        0.000011
Name: mass_rate*frequency, dtype: float64

**Donor of L-Arginine**

In [394]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Arginine")].value_counts("family_donor")

family_donor
f_Desulfovibrionaceae    43
f_Lactobacillaceae        8
f_Bacteroidaceae          5
Other                     1
f_Lachnospiraceae         1
dtype: int64

In [395]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Arginine")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
f_Lactobacillaceae       0.000777
f_Desulfovibrionaceae    0.000590
Other                    0.000030
f_Lachnospiraceae        0.000013
f_Bacteroidaceae         0.000009
Name: mass_rate*frequency, dtype: float64

In [396]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Arginine")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver      
CD_A       f_Fibrobacteraceae       0.000099
           f_Treponemataceae        0.000061
           f_Bacteroidaceae         0.000043
           Other                    0.000036
           f_Sphaerochaetaceae      0.000023
           f_Lachnospiraceae        0.000011
           f_Desulfovibrionaceae    0.000007
CD_P       f_Dysgonomonadaceae      0.000013
CM_A       Other                    0.000039
           f_Verruco-01             0.000017
           f_Treponemataceae        0.000011
           f_Dysgonomonadaceae      0.000010
           f_Desulfovibrionaceae    0.000009
           f_Bacteroidaceae         0.000008
           f_CAG-74                 0.000007
CM_P       Other                    0.000037
           f_Desulfovibrionaceae    0.000037
           f_Bacteroidaceae         0.000037
           f_Sphaerochaetaceae      0.000029
           f_Lentimicrobiaceae      0.000025
           f_Verruco-01             0.000021
           f_Desulfobu

In [397]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Arginine")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor         
CD_A       f_Desulfovibrionaceae    0.000279
CD_P       f_Lachnospiraceae        0.000013
CM_A       f_Desulfovibrionaceae    0.000091
           f_Bacteroidaceae         0.000009
CM_P       f_Desulfovibrionaceae    0.000220
CM_X       f_Lactobacillaceae       0.000777
M_P        Other                    0.000030
Name: mass_rate*frequency, dtype: float64

#### L-Lysine

**Receiver of L-Lysine**

In [398]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lysine")].value_counts("family_receiver")

family_receiver
Other                    19
f_Desulfovibrionaceae    13
f_Bacteroidaceae          7
f_Dysgonomonadaceae       6
f_Sphaerochaetaceae       6
f_CAG-74                  5
f_Lachnospiraceae         5
f_Treponemataceae         4
f_Clostridiaceae          2
f_Desulfobulbaceae        2
f_Lactobacillaceae        2
dtype: int64

In [399]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lysine")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
Other                    0.000389
f_Desulfovibrionaceae    0.000151
f_Sphaerochaetaceae      0.000123
f_CAG-74                 0.000111
f_Bacteroidaceae         0.000108
f_Lachnospiraceae        0.000062
f_Desulfobulbaceae       0.000048
f_Dysgonomonadaceae      0.000033
f_Treponemataceae        0.000024
f_Lactobacillaceae       0.000012
f_Clostridiaceae         0.000010
Name: mass_rate*frequency, dtype: float64

**Donor of L-Lysine**

In [400]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lysine")].value_counts("family_donor")

family_donor
f_Desulfovibrionaceae    31
f_Treponemataceae        20
f_Fibrobacteraceae       10
f_Bacteroidaceae          5
f_Lactobacillaceae        5
dtype: int64

In [401]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lysine")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
f_Desulfovibrionaceae    0.000526
f_Treponemataceae        0.000321
f_Lactobacillaceae       0.000189
f_Fibrobacteraceae       0.000027
f_Bacteroidaceae         0.000009
Name: mass_rate*frequency, dtype: float64

In [402]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lysine")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver      
CD_A       f_Bacteroidaceae         0.000034
           f_Sphaerochaetaceae      0.000027
           f_Lachnospiraceae        0.000025
           f_Treponemataceae        0.000024
           Other                    0.000021
           f_Desulfovibrionaceae    0.000006
CD_P       Other                    0.000093
           f_Desulfovibrionaceae    0.000073
           f_Bacteroidaceae         0.000032
           f_Desulfobulbaceae       0.000031
           f_Sphaerochaetaceae      0.000030
           f_Dysgonomonadaceae      0.000025
           f_Lachnospiraceae        0.000020
CM_A       f_Desulfovibrionaceae    0.000016
           Other                    0.000009
           f_Bacteroidaceae         0.000006
           f_CAG-74                 0.000006
           f_Dysgonomonadaceae      0.000003
CM_P       f_CAG-74                 0.000105
           f_Sphaerochaetaceae      0.000065
           f_Desulfovibrionaceae    0.000055
           f_Bacteroid

In [403]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lysine")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor         
CD_A       f_Desulfovibrionaceae    0.000115
           f_Fibrobacteraceae       0.000023
CD_P       f_Treponemataceae        0.000304
CM_A       f_Treponemataceae        0.000017
           f_Desulfovibrionaceae    0.000010
           f_Bacteroidaceae         0.000009
           f_Fibrobacteraceae       0.000003
CM_P       f_Desulfovibrionaceae    0.000318
CM_X       f_Lactobacillaceae       0.000189
M_P        f_Desulfovibrionaceae    0.000083
Name: mass_rate*frequency, dtype: float64

#### Thymine

**Receiver of L-Thymine**

In [404]:
steadier_sample_cross[(steadier_sample_cross.compound=="Thymine C5H6N2O2")].value_counts("family_receiver")

family_receiver
f_Lachnospiraceae        9
Other                    8
f_CAG-74                 3
f_Sphaerochaetaceae      3
f_Desulfovibrionaceae    2
dtype: int64

In [405]:
steadier_sample_cross[(steadier_sample_cross.compound=="Thymine C5H6N2O2")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
f_Lachnospiraceae        0.000019
f_CAG-74                 0.000014
Other                    0.000005
f_Sphaerochaetaceae      0.000004
f_Desulfovibrionaceae    0.000003
Name: mass_rate*frequency, dtype: float64

**Donor of Thymine**

In [406]:
steadier_sample_cross[(steadier_sample_cross.compound=="Thymine C5H6N2O2")].value_counts("family_donor")

family_donor
Other                    11
f_Desulfovibrionaceae     8
f_Fibrobacteraceae        3
f_Lachnospiraceae         2
f_Clostridiaceae          1
dtype: int64

In [407]:
steadier_sample_cross[(steadier_sample_cross.compound=="Thymine C5H6N2O2")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
f_Desulfovibrionaceae    2.721546e-05
Other                    1.238919e-05
f_Lachnospiraceae        2.534990e-06
f_Fibrobacteraceae       2.111755e-06
f_Clostridiaceae         1.521961e-07
Name: mass_rate*frequency, dtype: float64

In [408]:
steadier_sample_cross[(steadier_sample_cross.compound=="Thymine C5H6N2O2")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver      
CD_A       f_Lachnospiraceae        3.334903e-06
CD_P       Other                    3.877135e-06
           f_Desulfovibrionaceae    3.036685e-06
           f_Lachnospiraceae        1.485775e-06
           f_Sphaerochaetaceae      5.755451e-07
CD_X       f_Lachnospiraceae        1.636116e-06
CM_A       f_CAG-74                 1.136637e-06
CM_P       f_CAG-74                 1.245267e-05
           f_Sphaerochaetaceae      3.190788e-06
           Other                    7.530478e-07
M_P        f_Lachnospiraceae        1.236434e-05
           Other                    4.077591e-07
M_X        Other                    1.521961e-07
Name: mass_rate*frequency, dtype: float64

In [409]:
steadier_sample_cross[(steadier_sample_cross.compound=="Thymine C5H6N2O2")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor         
CD_A       f_Desulfovibrionaceae    2.359785e-06
           f_Fibrobacteraceae       9.751177e-07
CD_P       Other                    8.975140e-06
CD_X       Other                    1.636116e-06
CM_A       f_Fibrobacteraceae       1.136637e-06
CM_P       f_Desulfovibrionaceae    1.461857e-05
           Other                    1.777932e-06
M_P        f_Desulfovibrionaceae    1.023711e-05
           f_Lachnospiraceae        2.534990e-06
M_X        f_Clostridiaceae         1.521961e-07
Name: mass_rate*frequency, dtype: float64

In [410]:
steadier_sample_cross[steadier_sample_cross.family_receiver=="f_Desulfovibrionaceae"].groupby(["super_class","compound"]).sum()["mass_rate*frequency"].sort_values()

super_class                  compound                                                     
gases                        Hydrogen sulfide                                                 4.302194e-07
                             Hydrogen                                                         7.982087e-07
amino acids and derivatives  L-Tyrosine                                                       1.678141e-06
                             L-Phenylalanine                                                  1.901352e-06
alcohols and aldehydes       Glycolate C2H3O3                                                 1.945897e-06
nucleotides and derivatives  Thymine C5H6N2O2                                                 3.036685e-06
carboxylic acids and anions  Citrate                                                          5.992932e-06
                             2-Oxobutanoate                                                   6.851001e-06
polyamine                    Putrescine              

In [411]:
steadier_sample_cross[steadier_sample_cross.family_donor=="f_Desulfovibrionaceae"].groupby(["super_class","compound"]).sum()["mass_rate*frequency"].sort_values()

super_class                  compound                                             
alcohols and aldehydes       Glycolate C2H3O3                                         1.899621e-08
amino acids and derivatives  D-Alanine                                                4.775389e-07
carboxylic acids and anions  Fumarate                                                 6.265444e-06
other                        N-Acetyl-D-glucosamine(anhydrous)N-Acetylmuramic acid    1.446628e-05
gases                        Hydrogen sulfide                                         1.572672e-05
nucleotides and derivatives  Hypoxanthine                                             2.186543e-05
                             Thymine C5H6N2O2                                         2.721546e-05
other                        Indole                                                   2.824892e-05
alcohols and aldehydes        R  Acetoin C4H8O2                                       5.706611e-05
carboxylic acids and anion

#### D-Glyceraldehyde

**Receiver of D-Glyceraldehyde**

In [412]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Glyceraldehyde")].value_counts("family_receiver")

family_receiver
Other                    3
f_Desulfovibrionaceae    3
f_Lachnospiraceae        2
f_Clostridiaceae         1
dtype: int64

In [413]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Glyceraldehyde")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
f_Clostridiaceae         0.003009
Other                    0.002385
f_Lachnospiraceae        0.001192
f_Desulfovibrionaceae    0.000737
Name: mass_rate*frequency, dtype: float64

**Donor of D-Glyceraldehyde**

In [414]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Glyceraldehyde")].value_counts("family_donor")

family_donor
f_Lachnospiraceae     4
Other                 3
f_Clostridiaceae      1
f_Lactobacillaceae    1
dtype: int64

In [415]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Glyceraldehyde")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
f_Lactobacillaceae    0.003009
f_Lachnospiraceae     0.002659
Other                 0.001233
f_Clostridiaceae      0.000422
Name: mass_rate*frequency, dtype: float64

In [416]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Glyceraldehyde")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver      
CD_A       f_Desulfovibrionaceae    0.000401
CD_P       f_Lachnospiraceae        0.000897
           f_Desulfovibrionaceae    0.000336
CD_X       Other                    0.001905
CM_X       f_Clostridiaceae         0.003009
M_P        f_Lachnospiraceae        0.000295
           Other                    0.000058
M_X        Other                    0.000422
Name: mass_rate*frequency, dtype: float64

In [417]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Glyceraldehyde")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor      
CD_A       f_Lachnospiraceae     0.000401
CD_P       Other                 0.001233
CD_X       f_Lachnospiraceae     0.001905
CM_X       f_Lactobacillaceae    0.003009
M_P        f_Lachnospiraceae     0.000353
M_X        f_Clostridiaceae      0.000422
Name: mass_rate*frequency, dtype: float64